In [6]:
import os
import requests
from supabase import create_client, Client
from dotenv import load_dotenv
from datetime import datetime, timedelta

In [7]:
# Load environment variables
load_dotenv()

# Initialize clients
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
GROQ_API_URL = "https://api.groq.com/openai/v1/chat/completions"

supabase_url = os.getenv('SUPABASE_URL')
supabase_key = os.getenv('SUPABASE_ANON_KEY')
supabase: Client = create_client(supabase_url, supabase_key)


In [8]:
def get_upcoming_events(days_ahead=7, limit=50):
    """
    Fetch upcoming events from Supabase
    """
    try:
        # Calculate date range
        start_date = datetime.now().strftime('%Y-%m-%d')
        end_date = (datetime.now() + timedelta(days=days_ahead)).strftime('%Y-%m-%d')
        
        # Query Supabase using correct column names
        result = supabase.table('resident_advisor')\
            .select('*')\
            .gte('Date', start_date)\
            .lte('Date', end_date)\
            .limit(limit)\
            .execute()
        
        return result.data
    except Exception as e:
        print(f"Error fetching events: {e}")
        return []


In [9]:
def format_events_for_llm(events):
    """
    Format events data for the LLM prompt
    """
    if not events:
        return "No events found for the specified dates."
    
    formatted_events = []
    for event in events[:30]:  # Limit to avoid token limits
        # Using correct column names: Artists, Venue, Event name, Date
        event_text = f"• {event.get('Event name', 'N/A')}"
        if event.get('Date'):
            event_text += f" - {event.get('Date')}"
        if event.get('Venue'):
            event_text += f" at {event.get('Venue')}"
        if event.get('Artists'):
            event_text += f" | Artists: {event.get('Artists')}"
        formatted_events.append(event_text)
    
    return "\n".join(formatted_events)


In [10]:
def generate_recommendations(user_request, events_data):
    """
    Use Groq to generate event recommendations
    """
    prompt = f"""You are a helpful event recommender. A user has made this request: "{user_request}"

Here are the upcoming electronic music events in London:

{events_data}

Based on the user's request, please recommend 3-5 of the most suitable events. For each recommendation:
- Include the event name, date, and venue
- Briefly explain why it matches their request
- Keep it conversational and enthusiastic
- If no events perfectly match, recommend the closest alternatives

Format your response as a friendly text message (like an SMS)."""

    try:
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {GROQ_API_KEY}"
        }
        
        data = {
            "model": "meta-llama/llama-4-scout-17b-16e-instruct",
            "messages": [
                {"role": "system", "content": "You are a knowledgeable electronic music event recommender."},
                {"role": "user", "content": prompt}
            ],
            "max_tokens": 500,
            "temperature": 0.7
        }
        
        response = requests.post(GROQ_API_URL, headers=headers, json=data)
        response.raise_for_status()
        
        return response.json()["choices"][0]["message"]["content"]
    except Exception as e:
        return f"Error generating recommendations: {e}"


In [11]:
def main():
    """
    Main function to test the recommender
    """
    print("🎵 Event Recommender Test 🎵\n")
    
    # Test user requests
    test_requests = [
        "I want to see some techno this weekend",
        "Looking for house music events next week",
        "What's on tonight?",
        "I like melodic electronic music"
    ]
    
    # Fetch events
    print("Fetching events from database...")
    events = get_upcoming_events()
    print(f"Found {len(events)} events\n")
    
    if not events:
        print("No events found. Check your database connection and data.")
        return
    
    # Format events for LLM
    events_text = format_events_for_llm(events)
    
    # Test each request
    for i, request in enumerate(test_requests, 1):
        print(f"--- Test {i}: '{request}' ---")
        
        recommendation = generate_recommendations(request, events_text)
        print(f"Recommendation:\n{recommendation}\n")
        print("-" * 50 + "\n")


In [ ]:
if __name__ == "__main__":
    main()

🎵 Event Recommender Test 🎵

Fetching events from database...
Found 50 events

--- Test 1: 'I want to see some techno this weekend' ---
Recommendation:
"Hey! I've got some awesome techno event recs for you this weekend! 

1. PARADOX on 27th Aug at XOYO - This one's a great fit! It's a techno event featuring D1 x Samir (DVS), DEFILE, and Suly Aslan.

2. OPEN DECKS on 27th Aug at Planet Wax - This event is all about techno and seems like a fantastic opportunity to discover new artists.

3. Techno In London - Open Decks & Social on 28th Aug at Club Makossa - This event is specifically focused on techno and features Carlos Martinez and JFOX.

4. Lolahol on 27th Aug at Ormside Projects - Although not explicitly labeled as techno, Lolahol often features techno and experimental sounds, so it might be worth checking out!

Hope you find something that gets you dancing!"

--------------------------------------------------

--- Test 2: 'Looking for house music events next week' ---
Recommendation: